1. (*) Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции [numpy.linalg.svd()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html)


In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

In [2]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
X.shape

(150, 4)

In [34]:
U,D,V = np.linalg.svd(X,full_matrices=False)

In [26]:
a = np.random.randn(9, 6) + 1j*np.random.randn(9, 6)

In [27]:
a.shape

(9, 6)

In [28]:
U,D,V = np.linalg.svd(a)

In [46]:
np.dot(U * D, V).shape

(150, 4)

In [51]:
eig_sum = sum(D)
var_exp = [(i / eig_sum) * 100 for i in sorted(D, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описвыаемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описвыаемая каждой из компонент 
[80.59340691495326, 14.916876798004958, 2.9067159767294926, 1.5830003103122972]
Кумулятивная доля дисперсии по компонентам 
[ 80.59340691  95.51028371  98.41699969 100.        ]


In [55]:
V.T

array([[-0.75110816,  0.2841749 ,  0.50215472,  0.32081425],
       [-0.38008617,  0.5467445 , -0.67524332, -0.31725607],
       [-0.51300886, -0.70866455, -0.05916621, -0.48074507],
       [-0.16790754, -0.34367081, -0.53701625,  0.75187165]])

In [56]:
W = np.hstack((V.T[0].reshape(4,1), V.T[1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[-0.75110816 -0.38008617]
 [ 0.2841749   0.5467445 ]
 [ 0.50215472 -0.67524332]
 [ 0.32081425 -0.31725607]]


In [58]:
Z = X.dot(W)

2. (*) Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [72]:
import matplotlib.pyplot as plt
%matplotlib inline
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np
y = iris.target
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor





In [65]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25,random_state=42)

In [82]:
X_train1, X_test1, y_train1, y_test1 = model_selection.train_test_split(Z, y, test_size=0.25,random_state=42)

In [67]:
def gb_predict(X, trees_list, coef_list, eta):
    # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
    # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
    return np.array([sum([eta* coef * alg.predict([x])[0] for alg, coef in zip(trees_list, coef_list)]) for x in X])

In [68]:
def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)

In [69]:
def bias(y, z):
    return (y - z)

In [70]:
def gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta):
    
    # Деревья будем записывать в список
    trees = []
    
    # Будем записывать ошибки на обучающей и тестовой выборке на каждой итерации в список
    train_errors = []
    test_errors = []
    
    for i in range(n_trees):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)

        # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
        # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
        if len(trees) == 0:
            # обучаем первое дерево на обучающей выборке
            tree.fit(X_train, y_train)
            
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, coefs, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, coefs, eta)))
        else:
            # Получим ответы на текущей композиции
            target = gb_predict(X_train, trees, coefs, eta)
            
            # алгоритмы начиная со второго обучаем на сдвиг
            tree.fit(X_train, bias(y_train, target))
            
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, coefs, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, coefs, eta)))

        trees.append(tree)
        
    return trees, train_errors, test_errors

In [83]:
# Число деревьев в ансамбле
n_trees = 3

# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees

# Максимальная глубина деревьев
max_depth = 1

# Шаг
eta = 1

trees, train_errors, test_errors = gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta)

In [84]:
def evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta):
    train_prediction = gb_predict(X_train, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тренировочной выборке: {mean_squared_error(y_train, train_prediction)}')

    test_prediction = gb_predict(X_test, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тестовой выборке: {mean_squared_error(y_test, test_prediction)}')

In [85]:
evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta)

Ошибка алгоритма из 3 деревьев глубиной 1     с шагом 1 на тренировочной выборке: 0.05297506114803795
Ошибка алгоритма из 3 деревьев глубиной 1     с шагом 1 на тестовой выборке: 0.015413382648690044


In [86]:
# Число деревьев в ансамбле
n_trees = 3

# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees

# Максимальная глубина деревьев
max_depth = 1

# Шаг
eta = 1

trees, train_errors, test_errors = gb_fit(n_trees, max_depth, X_train1, X_test1, y_train1, y_test1, coefs, eta)
evaluate_alg(X_train1, X_test1, y_train1, y_test1, trees, coefs, eta)

Ошибка алгоритма из 3 деревьев глубиной 1     с шагом 1 на тренировочной выборке: 0.0568663007958561
Ошибка алгоритма из 3 деревьев глубиной 1     с шагом 1 на тестовой выборке: 0.052880078499875596


С применением PCA на тестовой выборки результат лучше чем без него 